In [ ]:
!pip install bs4

In [ ]:
# import os
import sys
import pandas as pd
import numpy as np
import json
import itertools
import threading
import time
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page

In [ ]:
username = ''

In [ ]:
done = False
#here is the animation
def animate():
    for c in itertools.cycle(['|', '/', '-', '\\']):
        if done:
            break
        sys.stdout.write('\rloading ' + c)
        sys.stdout.flush()
        time.sleep(0.1)
    sys.stdout.write('\rDone!     ')

t = threading.Thread(target=animate)
t.start()

#long process here
def my_films(username):
    
    url = requests.get(f'https://letterboxd.com/{username}/films/')
    html = url.text
    html = BeautifulSoup(html, 'html5lib')

    def number_of_pages(html):
        number_of_pages = html.find_all('li', class_="paginate-page")
        x = number_of_pages[-1]
        x = str(x)
        x = x.split('page/')[1]
        x = x.split('/">')[0]
        x = int(x)
        return x

    number_of_pages = number_of_pages(html)

    list_of_films = []
    my_ratings = []
    minutes_list = []
    year_list = []
    director_list = []
    ratingCount_list = []
    reviewCount_list = []
    average_rating_list = []
    film_description_list = []
    genre_list = []
    actors_list = []
    countryOfOrigin_list = []

    def get_film_data(html):
        
        title_names = html.find_all('li', class_="poster-container")
        num_of_title_names_at_page = len(title_names)
        ratings = html.find_all('p', class_="poster-viewingdata -rated-and-liked")

        for i in range(0, num_of_title_names_at_page):
            title = str(title_names[i])
            title_ = title.split('<img alt="')[1]
            title_name = title_.split('" class=')[0]
            list_of_films.append(title_name)

            rating = ratings[i]
            rating = str(rating)
            if rating.find('★') != -1:
                idx = rating.find('★')
                rating = rating[idx:]
                rating = rating.split('</span>')[0]
                no_of_stars = rating.count('★')
                half_star = rating.count('½') * 0.5
                rating = no_of_stars + half_star
            else:
                rating = np.nan
            my_ratings.append(rating)

            film_path = title
            film_path = film_path.split('data-target-link="')[1]
            film_path = film_path.split('" data-target-link-target=')[0]
            url2 = requests.get(f'https://letterboxd.com/{film_path}/')
            html2 = url2.text
            html2 = BeautifulSoup(html2, 'html5lib') 

            film_info = html2.find_all('script')
            film_info = str(film_info)
            film_info = film_info.split('/* <![CDATA[ */\n')[1]
            film_info = film_info.split('\n/* ]]> */')[0]
            film_info = json.loads(film_info)

            minutes = html2.find_all('p', class_="text-link text-footer")
            minutes = str(minutes)
            minutes = minutes.replace('\n','')
            minutes = minutes.replace('\t','')
            minutes = minutes.split('[<p class="text-link text-footer">')[1]
            minutes = minutes.split('\xa0mins')[0]
            if minutes.find(',') != -1:
                minutes = minutes.replace(',','')
            minutes = int(minutes)
            minutes_list.append(minutes)

            year = film_info['releasedEvent'][0]['startDate']
            year_list.append(year)
            
            if 'director' in film_info.keys():
                if len(film_info['director']) == 1:
                    director = film_info['director'][0]['name']
                else:
                    directors_in_this_film = []
                    for j in range(0, len(film_info['director'])):
                        directors_in_this_film.append(film_info['director'][j]['name'])
                        director = directors_in_this_film
            else:
                director = np.nan
            director_list.append(director)
            
            if 'aggregateRating' in film_info.keys():
                if 'ratingCount' in film_info['aggregateRating']:
                    ratingCount = film_info['aggregateRating']['ratingCount']
                else:
                    ratingCount = np.nan
                if 'reviewCount' in film_info['aggregateRating']:
                    reviewCount = film_info['aggregateRating']['reviewCount']
                else:
                    reviewCount = np.nan
                if 'ratingValue' in film_info['aggregateRating']:
                    average_rating = film_info['aggregateRating']['ratingValue']
                else:
                    average_rating = np.nan
            else:
                ratingCount = np.nan
                reviewCount = np.nan
                average_rating = np.nan
            ratingCount_list.append(ratingCount)
            reviewCount_list.append(reviewCount)
            average_rating_list.append(average_rating)
            
            if 'genre' in film_info.keys():
                genre = film_info['genre']
            else:
                genre = np.nan
            genre_list.append(genre)

            if 'actors' in film_info.keys():
                if len(film_info['actors']) == 1:
                    actors = film_info['actors'][0]['name']
                else:
                    actors_in_this_film = []
                    for j in range(0, len(film_info['actors'])):
                        actors_in_this_film.append(film_info['actors'][j]['name'])
                        actors = actors_in_this_film
            else:
                actors = np.nan
            actors_list.append(actors)

            if 'countryOfOrigin' in film_info.keys():
                if len(film_info['countryOfOrigin']) == 1:
                    country = film_info['countryOfOrigin'][0]['name']
                else:
                    countries_this_film = []
                    for j in range(0, len(film_info['countryOfOrigin'])):
                        countries_this_film.append(film_info['countryOfOrigin'][j]['name'])
                        country = countries_this_film
            else:
                country = np.nan
            countryOfOrigin_list.append(country)

            film_description = html2.find_all('div', class_="truncate")
            film_description = str(film_description)
            if film_description.find('<p>') != -1:
                film_description = film_description.split('<p>')
                if len(film_description) > 0:
                    film_description = film_description[1]
                    film_description = film_description.split('</p>')[0]
                else:
                    film_description = np.nan
            else:
                    film_description = np.nan
            film_description_list.append(film_description)

        return     list_of_films, my_ratings, year_list, director_list, ratingCount_list, reviewCount_list, average_rating_list, film_description_list, genre_list, actors_list, countryOfOrigin_list, minutes_list

    list_of_films, my_ratings, year_list, director_list, ratingCount_list, reviewCount_list, average_rating_list, film_description_list, genre_list, actors_list, countryOfOrigin_list, minutes_list = get_film_data(html)

    for i in range(2, number_of_pages +1):
        url3 = requests.get(f'https://letterboxd.com/{username}/films/page/{i}/')
        html3 = url3.text
        html3 = BeautifulSoup(html3, 'html5lib')
    
        list_of_films, my_ratings, year_list, director_list, ratingCount_list, reviewCount_list, average_rating_list, film_description_list, genre_list, actors_list, countryOfOrigin_list, minutes_list = get_film_data(html3)

    data = {'Films':list_of_films,
            'Minutes': minutes_list,
            'Year':year_list,
            'Director':director_list,
            'My Ratings':my_ratings,
            'Average Ratings':average_rating_list,
            'Genres':genre_list,
            'Actors':actors_list,
            'Description':film_description_list,
            'Countries':countryOfOrigin_list,
            'RatingsCount':ratingCount_list,
            'ReviewCount':reviewCount_list}
    
    df = pd.DataFrame(data)
    df.reset_index(drop=True, inplace=True)

    def list_to_str(x):
        if x == x:
            if type(x) != list:
                x = str(x)
            if type(x) == list:
                outputs =[]
                for j in range(0, len(x)):
                    outputs.append(x[j])
                x = ', '.join(outputs)
        return x

    df['Countries'] = df['Countries'].apply(lambda x: list_to_str(x))
    df['Director'] = df['Director'].apply(lambda x: list_to_str(x))
    df['Genres'] = df['Genres'].apply(lambda x: list_to_str(x))
    df['Actors'] = df['Actors'].apply(lambda x: list_to_str(x))

    return df

df = my_films(f'{username}')

time.sleep(10)
done = True

df

In [ ]:
df.to_csv('', encoding='utf-8', index=False)